In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по молоку.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Молоко
203,АКТЮБИНСКАЯ ОБЛАСТЬ,2021-12-01,18772.70
1783,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2019-04-01,31724.10
1721,ОБЛАСТЬ ҰЛЫТАУ,2024-02-01,2505.80
1591,МАНГИСТАУСКАЯ ОБЛАСТЬ,2021-01-01,110.20
1876,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2017-01-01,15464.60
1528,МАНГИСТАУСКАЯ ОБЛАСТЬ,2015-10-01,272.80
259,АЛМАТИНСКАЯ ОБЛАСТЬ,2016-08-01,68208.20
966,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2018-12-01,24142.30
1768,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2018-01-01,15285.00
1893,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2018-06-01,75459.30


In [3]:
regions = df['Регион'].unique()
target   = "Молоко"
horizon  = 3
epsilon = 1e-6

In [4]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [5]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[8444.94, 9839.59, 15952.39]","[10829.7, 11876.6, 17387.5]",1991.32,1952.29,15.81
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[11965.2, 19362.36, 23847.83]","[11876.6, 17387.5, 20221.1]",2384.75,1896.73,10.01
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[19247.39, 23707.56, 29991.84]","[17387.5, 20221.1, 24058.8]",4115.64,3759.79,17.53
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[21907.75, 27733.65, 31788.8]","[20221.1, 24058.8, 28277.0]",3092.04,2957.77,12.01
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[26034.58, 29857.09, 27484.33]","[24058.8, 28277.0, 26267.7]",1620.76,1590.83,6.14
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[53357.54, 42776.63, 39662.46]","[54824.2, 39643.3, 38641.4]",2082.58,1873.68,4.41
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[43935.52, 40722.43, 37328.09]","[39643.3, 38641.4, 35950.3]",2866.60,2583.68,6.68
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[36787.97, 33759.67, 29843.29]","[38641.4, 35950.3, 32936.5]",2435.98,2379.09,6.76
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[35442.75, 31314.24, 29350.76]","[35950.3, 32936.5, 30760.6]",1275.01,1179.88,3.64


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,1743.41,1586.41,8.12
1,АКТЮБИНСКАЯ ОБЛАСТЬ,1823.12,1653.50,10.90
2,АЛМАТИНСКАЯ ОБЛАСТЬ,3617.99,3393.85,13.73
3,АТЫРАУСКАЯ ОБЛАСТЬ,331.57,289.15,10.65
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2881.20,2521.94,12.90
5,ГАЛМАТЫ,24.20,20.12,68.94
6,ГАСТАНА,2.68,2.33,14.25
7,ГШЫМКЕНТ,656.96,606.91,9.83
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,747.18,673.19,3.73
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1414.09,1300.50,7.89


In [ ]:
# res_hw[res_hw['Регион'] == 'МАНГИСТАУСКАЯ ОБЛАСТЬ']

,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
130,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[-0.48, -0.43, -0.28]","[0.0, 0.0, 0.0]",0.41,0.40,inf
131,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[0.09, 0.35, 0.71]","[0.0, 0.0, 0.0]",0.46,0.38,inf
132,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[0.25, 0.58, 5.13]","[0.0, 0.0, 0.0]",2.99,1.99,inf
133,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[0.28, 3.97, 1.43]","[0.0, 0.0, 0.0]",2.44,1.89,inf
134,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[2.92, 0.92, 1.21]","[0.0, 0.0, 0.0]",1.90,1.68,inf
135,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[-0.49, -0.41, -0.53]","[0.0, 0.0, 0.0]",0.48,0.48,inf
136,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[0.14, -0.1, -0.13]","[0.0, 0.0, 0.0]",0.13,0.13,inf
137,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[-0.21, -0.23, -0.38]","[0.0, 0.0, 0.0]",0.28,0.27,inf
138,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[-0.04, -0.22, -0.42]","[0.0, 0.0, 0.0]",0.28,0.23,inf
139,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-10,2024-12,"[-0.19, -0.4, -0.34]","[0.0, 0.0, 0.0]",0.32,0.31,inf


## SARIMA

In [7]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(1, 1, 0)","(1, 1, 0, 12)",3883.57,3851.82,29.33,"[7344.14, 8359.17, 12835.03]","[10829.7, 11876.6, 17387.5]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(1, 1, 0)","(1, 1, 1, 12)",2413.25,1991.45,10.73,"[12157.07, 19465.54, 23836.95]","[11876.6, 17387.5, 20221.1]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(1, 1, 0)","(1, 1, 1, 12)",3371.51,3158.34,14.87,"[19117.23, 23347.32, 28677.89]","[17387.5, 20221.1, 24058.8]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(1, 1, 0)","(1, 1, 1, 12)",2301.28,2220.63,9.01,"[21654.74, 26385.66, 31178.4]","[20221.1, 24058.8, 28277.0]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(1, 1, 0)","(1, 1, 1, 12)",952.33,933.11,3.56,"[25005.19, 29436.37, 26961.26]","[24058.8, 28277.0, 26267.7]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 1, 0)","(0, 1, 2, 12)",2445.68,2106.31,4.91,"[53031.11, 43404.1, 39406.45]","[54824.2, 39643.3, 38641.4]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(0, 1, 0)","(0, 1, 2, 12)",3281.39,2529.50,6.44,"[44912.31, 40762.92, 35752.34]","[39643.3, 38641.4, 35950.3]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(1, 0, 0)","(0, 1, 2, 12)",2453.40,2394.94,6.81,"[36862.23, 33622.72, 29858.44]","[38641.4, 35950.3, 32936.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(1, 0, 0)","(0, 1, 2, 12)",1206.05,1142.41,3.50,"[35319.05, 31370.46, 29530.66]","[35950.3, 32936.5, 30760.6]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,1825.95,1705.76,9.30
1,АКТЮБИНСКАЯ ОБЛАСТЬ,1225.38,1164.08,8.22
2,АЛМАТИНСКАЯ ОБЛАСТЬ,4890.12,4317.59,16.55
3,АТЫРАУСКАЯ ОБЛАСТЬ,219.63,199.04,10.06
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,4213.97,3709.49,16.93
5,ГАЛМАТЫ,35.52,30.29,104.99
6,ГАСТАНА,2.20,1.93,12.35
7,ГШЫМКЕНТ,537.04,498.23,8.45
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,831.64,793.86,5.50
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1854.18,1746.65,9.18


## Facebook Prophet

In [8]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


11:46:40 - cmdstanpy - INFO - Chain [1] start processing
11:46:41 - cmdstanpy - INFO - Chain [1] done processing
11:46:41 - cmdstanpy - INFO - Chain [1] start processing
11:46:41 - cmdstanpy - INFO - Chain [1] done processing
11:46:41 - cmdstanpy - INFO - Chain [1] start processing
11:46:41 - cmdstanpy - INFO - Chain [1] done processing
11:46:42 - cmdstanpy - INFO - Chain [1] start processing
11:46:42 - cmdstanpy - INFO - Chain [1] done processing
11:46:42 - cmdstanpy - INFO - Chain [1] start processing
11:46:42 - cmdstanpy - INFO - Chain [1] done processing
11:46:42 - cmdstanpy - INFO - Chain [1] start processing
11:46:42 - cmdstanpy - INFO - Chain [1] done processing
11:46:42 - cmdstanpy - INFO - Chain [1] start processing
11:46:43 - cmdstanpy - INFO - Chain [1] done processing
11:46:43 - cmdstanpy - INFO - Chain [1] start processing
11:46:43 - cmdstanpy - INFO - Chain [1] done processing
11:46:43 - cmdstanpy - INFO - Chain [1] start processing
11:46:43 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,4194.15,4167.78,31.84,"[7214.59, 7751.32, 12624.56]","[10829.7, 11876.6, 17387.5]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,4073.55,4069.08,25.66,"[8033.05, 13333.75, 15911.15]","[11876.6, 17387.5, 20221.1]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,3613.57,3577.68,17.41,"[14239.76, 16926.94, 19767.66]","[17387.5, 20221.1, 24058.8]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,3780.85,3743.93,15.47,"[17190.79, 20144.61, 23989.73]","[20221.1, 24058.8, 28277.0]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,4496.18,4450.01,17.02,"[20206.47, 24121.17, 20925.82]","[24058.8, 28277.0, 26267.7]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,3662.43,3482.97,8.14,"[51822.94, 44689.8, 41042.55]","[54824.2, 39643.3, 38641.4]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,5152.59,4542.46,11.69,"[47145.69, 43221.08, 37495.62]","[39643.3, 38641.4, 35950.3]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,2593.94,1934.50,5.12,"[42954.92, 37179.56, 33197.22]","[38641.4, 35950.3, 32936.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,1277.98,1178.88,3.55,"[37513.93, 33418.82, 32251.28]","[35950.3, 32936.5, 30760.6]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,4211.87,4187.99,21.34
1,АКТЮБИНСКАЯ ОБЛАСТЬ,2976.40,2863.32,19.02
2,АЛМАТИНСКАЯ ОБЛАСТЬ,10870.58,10488.37,39.35
3,АТЫРАУСКАЯ ОБЛАСТЬ,815.70,777.67,27.55
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,8553.64,8401.11,41.14
5,ГАЛМАТЫ,15.43,11.76,37.92
6,ГАСТАНА,5.99,5.49,65.71
7,ГШЫМКЕНТ,822.96,760.33,12.39
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,3090.70,2983.14,14.85
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1442.51,1236.89,7.72


In [9]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW           MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     8.12                  9.30         21.34          HW
           АКТЮБИНСКАЯ ОБЛАСТЬ    10.90                  8.22         19.02      SARIMA
           АЛМАТИНСКАЯ ОБЛАСТЬ    13.73                 16.55         39.35          HW
            АТЫРАУСКАЯ ОБЛАСТЬ    10.65                 10.06         27.55      SARIMA
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    12.90                 16.93         41.14          HW
                       ГАЛМАТЫ    68.94                104.99         37.92     Prophet
                       ГАСТАНА    14.25                 12.35         65.71      SARIMA
                      ГШЫМКЕНТ     9.83                  8.45         12.39      SARIMA
            ЖАМБЫЛСКАЯ ОБЛАСТЬ     3.73                  5.50         14.85          HW
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     7.89                  9.18          7.72     Prophet
        КАРАГАНДИНСКАЯ ОБЛАСТЬ  